In [1]:
import sys
import os
sys.path.append("/Users/baeyeongmin/Desktop/workspace/dacon-2024-gbt-hackerton")
sys.path.append("/root/dacon-2024-gbt-hackerton")

In [3]:
from src import root_path
from src.data_processing import load_data
train_df, test_df = load_data(root_path=root_path)

In [8]:
import pygwalker as pyg

In [9]:
walker = pyg.walk(train_df[train_df['분류'] != '지역'])

Box(children=(HTML(value='<div id="ifr-pyg-0" style="height: auto">\n    <head>\n        <meta http-equiv="Con…

In [10]:
import re
def normalize_title(text):
    text = re.sub(r'\s+', ' ', text).strip()
    return text.strip()

def normalize_keywords(text):
    text = re.sub(r'[^가-힣\s,]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text.strip()

def clean_keywords(keywords):
    keywords = keywords.split(',')
    keywords = [keyword.strip() for keyword in keywords if keyword.strip()]
    return ' '.join(keywords)

train_df['title'] = train_df['제목'].apply(normalize_title)
train_df['keywords'] = train_df['키워드'].apply(lambda x: clean_keywords(normalize_keywords(x)))
train_df['text'] = train_df['title'] + ' [SEP] ' + train_df['keywords']

test_df['title'] = test_df['제목'].apply(normalize_title)
test_df['keywords'] = test_df['키워드'].apply(lambda x: clean_keywords(normalize_keywords(x)))
test_df['text'] = test_df['title'] + ' [SEP] ' + test_df['keywords']

In [11]:
def get_length(text):
    return len(text.split())

# 제목, 키워드, 전체 텍스트의 길이 계산
train_df['title_length'] = train_df['title'].apply(get_length)
train_df['keywords_length'] = train_df['keywords'].apply(get_length)
train_df['total_length'] = train_df['text'].apply(get_length)

DISPLAY = False 
if DISPLAY:
    # 길이 통계 출력
    print("제목 길이 통계:")
    print(train_df['title_length'].describe())
    print("\n키워드 길이 통계:")
    print(train_df['keywords_length'].describe())
    print("\n전체 텍스트 길이 통계:")
    print(train_df['total_length'].describe())

In [12]:
from src.data_processing import get_label_encoded

train_df, label_encoder = get_label_encoded(train_df)

## Class imbalance problem
### Training Stage
1. model 1: 지역=1, 지역이 아닌 라벨=0 으로 한 binanry classification model
2. model 2: 라벨이 0으로 classification model에 대해서 다시 나머지 55개 주제에 대해서 classification model(지역 라벨에 대한 로우들은 모두 제거한 데이터로 학습+Focal loss)

### inference Stage
- model 1 추론 후 model 2 추론하면 끝.


In [13]:
from src.data_processing import get_label_encoded

# 기존 레이블 인코딩 유지
train_df, label_encoder = get_label_encoded(train_df)
# 모델 1을 위한 이진 분류 레이블 생성
train_df['label_model1'] = (train_df['분류'] == '지역').astype(int)
# 모델 2를 위한 데이터셋 생성 (지역 제외)
train_df_model2 = train_df[train_df['분류'] != '지역'].copy()
train_df_model2['label_model2'] = train_df_model2['label']

DISPLAY = False 
if DISPLAY:
    # 결과 확인
    print("모델 1 데이터:")
    print(train_df[['분류', 'label', 'label_model1']].head(10))
    print("\n모델 1 레이블 분포:")
    print(train_df['label_model1'].value_counts())

    print("\n모델 2 데이터:")
    print(train_df_model2[['분류', 'label', 'label_model2']].head(10))
    print("\n모델 2 레이블 분포:")
    print(train_df_model2['label_model2'].value_counts())

    # 데이터셋 크기 확인
    print("\n전체 데이터셋 크기:", len(train_df))
    print("모델 2 데이터셋 크기:", len(train_df_model2))

In [21]:
secondary_df = train_df[train_df['분류'] != '지역'].copy()
primary_df = train_df.copy()
primary_df.loc[primary_df['분류'] != '지역', '분류'] = '비지역'


In [ ]:
from src.data_processing import split_data

tdf1, vdf1 = split_data(train_df, test_size=0.2, random_state=42, stratify_col='label_model1')
tdf2, vdf2 = split_data(train_df_model2, test_size=0.2, random_state=42, stratify_col='label_model2')

In [1]:
import os
os.getlogin()

'root'